In [ ]:
#%% General imports
import numpy as np
import torch

# DeepMoD stuff
from multitaskpinn import DeepMoD
from multitaskpinn.model.func_approx import Siren, NN
from multitaskpinn.model.library import Library1D
from multitaskpinn.model.constraint import LeastSquares
from multitaskpinn.model.sparse_estimators import Threshold 
from experiment_code import train, train_bayes_MSE_optim, train_bayes_full_optim
from deepymod_torch.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%config InlineBackend.figure_format = 'svg'

# Setting cuda
if torch.cuda.is_available():
    device ='cuda'
else:
    device = 'cpu'

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


%load_ext autoreload
%autoreload 2

In [ ]:
# Prepping data
data = loadmat('../kuramoto_sivishinky.mat')

t = data['tt']
x = data['x']
u = data['uu']

# Normalizing data
t = (t - t.min())/(t.max()-t.min()) * 2 - 1
x = (x - x.min())/(x.max()-x.min()) * 2 - 1

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')

# Limiting to non-chaotic part
lower_lim = 80
x_grid = x_grid[:, lower_lim:]
t_grid = t_grid[:, lower_lim:]
u = u[:, lower_lim:]

# %%Making training data
X = np.concatenate((t_grid.reshape(-1, 1), x_grid.reshape(-1, 1)), axis=1)
y = u.reshape(-1, 1)

# Adding noise
noise_level = 0.05
y_noisy = y + noise_level * np.std(y, axis=0) * np.random.randn(*y.shape)
number_of_samples = 25000

# Into tensor
idx = np.random.permutation(y.shape[0])
X_train = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32).to(device)
y_train = torch.tensor(y_noisy[idx, :][:number_of_samples], dtype=torch.float32).to(device)

In [ ]:
(noise_level * np.std(y, axis=0))**2

In [ ]:
plt.contourf(u)

# MSE , optim

In [6]:
# %%Configuring model
network = Siren(2, [50, 50, 50, 50, 50, 50, 50, 50], 1) # Function approximator
library =  Library1D(poly_order=1, diff_order=4) # Library function
estimator = Threshold(0.1)#PDEFIND(lam=1e-6, dtol=0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

# %% Setting schedulers
sparsity_scheduler = TrainTestPeriodic(patience=8, delta=1e-5)#Periodic(initial_epoch=10000, periodicity=100) # Defining when to apply sparsity
optimizer = torch.optim.Adam(model.parameters(), betas=(0.999, 0.999), lr=0.00025,  amsgrad=True) # Defining optimizer

In [ ]:
train_bayes_MSE_optim(model, X_train, y_train, optimizer, sparsity_scheduler, log_dir='data/bayes_mse')

# MSE + Reg

In [5]:
# %%Configuring model
network = NN(2, [50, 50, 50, 50, 50, 50, 50, 50], 1) # Function approximator
library =  Library1D(poly_order=1, diff_order=4) # Library function
estimator = Threshold(0.1)#PDEFIND(lam=1e-6, dtol=0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

# %% Setting schedulers
sparsity_scheduler = TrainTestPeriodic(patience=50000, delta=0.0)#Periodic(initial_epoch=10000, periodicity=100) # Defining when to apply sparsity
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), lr=1e-3,  amsgrad=True) # Defining optimizer

In [6]:
train_bayes_full_optim(model, X_train, y_train, optimizer, sparsity_scheduler, log_dir='data/correct_fast_6', max_iterations=50000, delta=0.0)

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
tensor([13850.3359], device='cuda:0', grad_fn=<SubBackward0>) tensor([[-129379.4453]], device='cuda:0', grad_fn=<MulBackward0>)
          0      0.00%               0s   -1.16e+05   1.47e+00   2.52e-08   8.93e+01 tensor([13957.6094], device='cuda:0', grad_fn=<SubBackward0>) tensor([[-126674.1328]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([14058.9492], device='cuda:0', grad_fn=<SubBackward0>) tensor([[350478.3750]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([14195.8223], device='cuda:0', grad_fn=<SubBackward0>) tensor([[1.1295e+08]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([14150.5176], device='cuda:0', grad_fn=<SubBackward0>) tensor([[137243.2656]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([14112.2373], device='cuda:0', grad_fn=<SubBackward0>) tensor([[1.1182e+08]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([14035.3223], device='cuda:0', grad_fn=<SubBackward0>) ten

KeyboardInterrupt: 

In [12]:
data_train = X_train.to('cpu')
target_train = y_train.to('cpu')

In [14]:
prediction, time_derivs, thetas = model(data_train).to('cpu')


t = time_derivs[0]
Theta = thetas[0]

tau_MSE = torch.exp(-model.s[:, 0])
alpha = model.s[:, 1]#torch.exp(model.s[:, 1]) #1 / MSE[0].data
beta = model.s[:, 2]#torch.exp(model.s[:, 2]) #torch.tensor(1e-5).to(Theta.device)

M = thetas[0].shape[1]

Sn = torch.inverse(torch.eye(M).to(Theta.device) * alpha + beta * Theta.T @ Theta)
mn = beta * Sn @ Theta.T @ t
tau_reg = torch.trace(torch.log(1/beta + Theta @ Sn @ Theta.T))

MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
Reg = torch.mean((t - Theta @ mn)**2, dim=0)


loss = torch.sum(tau_MSE * MSE - torch.log(tau_MSE) + tau_reg * Reg - torch.log(tau_reg)) 

RuntimeError: Expected object of device type cuda but got device type cpu for argument #2 'mat1' in call to _th_addmm

In [15]:
tau_MSE

tensor([0.8754], device='cuda:0', grad_fn=<ExpBackward>)

In [11]:
tau_reg

NameError: name 'tau_reg' is not defined

In [11]:
model.s[:, 0]

tensor([0.0198], device='cuda:0', grad_fn=<SelectBackward>)

In [17]:
prediction, time_derivs, thetas = model(data_train)

MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
Reg = torch.stack([torch.mean((dt - theta @ coeff_vector)**2)
                   for dt, theta, coeff_vector in zip(time_derivs, thetas, model.constraint_coeffs(scaled=False, sparse=True))])

t = time_derivs[0]
Theta = thetas[0]

alpha = 1 / MSE[0].data
beta = torch.tensor(1e-5).to(Theta.device)

M = thetas[0].shape[1]
N = thetas[0].shape[0]
A = torch.eye(M).to(Theta.device) * alpha + beta * Theta.T @ Theta
mn = beta * torch.inverse(A) @ Theta.T @ t
E_mn = beta / 2 * torch.sum((t - Theta @ mn)**2) + alpha / 2 * mn.T @ mn
reg_loss = -(M/(2*N) * torch.log(alpha) + 1/2 * torch.log(beta) - E_mn / N - 1/(2* N) * torch.trace(torch.log(A)))

loss = torch.sum(torch.log(MSE) + reg_loss) 

In [18]:
loss

tensor(6.1267, device='cuda:0', grad_fn=<SumBackward0>)

In [19]:
reg_loss

tensor([[5.7576]], device='cuda:0', grad_fn=<NegBackward>)

In [21]:
torch.log(MSE)

tensor([0.3691], device='cuda:0', grad_fn=<LogBackward>)

In [15]:
E_mn

tensor([[0.0046]], device='cuda:0', grad_fn=<AddBackward0>)

In [16]:
 N/2 * torch.log(beta)

tensor(-143911.5625, device='cuda:0')